In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
def get_html(url):
    r = requests.get('http://localhost:8050/render.html',params={'url':url,'wait':2})
    html = BeautifulSoup(r.text,'html.parser')
    return html

In [9]:
def clean_review(produrl): 
    reviewList = []
    for j in ["one","two","three","four","five"]:
        #print(j)
        url = produrl+"&filterByStar="+j+"_star"
        prodHTML = get_html(url)
        product = prod[0]
        while True:
            reviews = prodHTML.find_all("div",{"class":"a-section review aok-relative"})
            for review in reviews:
                rating = review.find("a",{"data-hook":"review-title"}).find_all("span")[0].text
                title = review.find("a",{"data-hook":"review-title"}).find_all("span")[2].text
                review = review.find("span",{"data-hook":"review-body"}).text.replace("\n","")
                reviewList.append({"Rating":rating,"Title":title,"Review":review})
            if prodHTML.find("li",{"class":"a-disabled a-last"}) == None and prodHTML.find("li",{"class":"a-last"})!=None:
                url = ["https://www.amazon.com/"+a["href"] for a in prodHTML.find("li",{"class":"a-last"})][0]
                r = requests.get('http://localhost:8050/render.html',params={'url':url,'wait':2})
                prodHTML = BeautifulSoup(r.text,'html.parser')
            else:
                break
    return {"Product":product,"Reviews":[reviewList]}

In [5]:
url = 'https://www.amazon.com/s?k=health+and+wellness&s=exact-aware-popularity-rank&crid=2YGI85R24J80V&qid=1707073286&sprefix=health+and+wellnes%2Caps%2C165&ref=sr_st_exact-aware-popularity-rank&ds=v1%3Aq%2FAEeobgajciOnlVubobHYLDJ9X91%2FT05SbD5OU1Amk'
healthHTML = get_html(url)

categories = healthHTML.find("span",{"data-component-type":"s-search-results"})

In [6]:
hrefDF = pd.DataFrame(columns = ["Product","Link"])
prodList = []

for a in categories.find_all("a",href=True):
    newLink = "https://www.amazon.com/"+a["href"]
    if "/dp/" in newLink:
        newLink = newLink.replace("/dp/","/product-reviews/",1)
        sub1 = "amazon.com//"
        sub2 = "/product-reviews/"
        idx1 = newLink.index(sub1)
        idx2 = newLink.index(sub2)
        prod = newLink[idx1 + len(sub1): idx2]
        if "product-reviews" in newLink and (prodList is None or prod not in prodList):
            prodList.append(prod)
            hrefDF = pd.concat([hrefDF,pd.DataFrame({"Product":[prod],"Link":[newLink]})])

#finalHrefList = []
#[finalHrefList.append(i) for i in hrefList if i not in finalHrefList and "#customerReviews" not in i]

masterReviews = pd.DataFrame(columns = ["Product","Reviews"]);


In [15]:
# First 5 products scraped 
# if want all 40 urls, replace 'i in range(0,5)' with 'url in hrefList' and 'hrefList[i]' with 'url'
for i in range(0,5):
    masterReviews = pd.concat([masterReviews,pd.DataFrame(clean_review3(hrefDF.iloc[i,:]))])
masterReviews = masterReviews.reset_index(drop=True)   

In [70]:
for i in range(0,4):
    product = masterReviews.iloc[i,:][0]
    reviews = pd.DataFrame(masterReviews.iloc[i,:]["Reviews"])
    product = product.replace("/Day"," Day")
    reviews.to_csv(product+".csv")

Potential way to improve speed of program (consider parallel processing)

In [ ]:
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        clean_review(response.text())
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        response1 = await fetch(session, produrl+"&filterByStar=one_star")
        response2 = await fetch(session, produrl+"&filterByStar=two_star")
        response3 = await fetch(session, produrl+"&filterByStar=three_star")
        response4 = await fetch(session, produrl+"&filterByStar=four_star")
        response5 = await fetch(session, produrl+"&filterByStar=four_star")

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

Set aside selenium for now

https://www.youtube.com/watch?v=daILiXCLom8&t=402s

https://www.youtube.com/watch?v=xrYDlx8evR0

https://hub.docker.com/r/selenium/standalone-chrome

https://stackoverflow.com/questions/66662820/m1-docker-preview-and-keycloak-images-platform-linux-amd64-does-not-match-th

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(
command_executor='http://localhost:4444/wd/hub',
options=options
)

driver.get(finalHrefList[0])
print(driver.get_attribute("outerHTML"))
#buttonTable=driver.find_element("xpath", "/html/body/div[1]/main/div[1]/div[3]/div/div[1]/div/div/div[2]/a")
#buttonTable.click()

#buttonDrop = driver.find_element("xpath","/html/body/div[1]/main/div[1]/div[3]/div/div[2]/div/div[1]/div/button")
#buttonDrop.click()

ProtocolError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))